# Extract paragraphs, combine short paragraphs

In [2]:
import re
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import random


[nltk_data] Downloading package punkt to /Users/yuchenluo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# UCSB New Loop

In [38]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            # loop through all paragraphs inside each speech 
            i = 0    
            while i < len(pars):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars):
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                elif i == (len(pars) - 1): 
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1    

In [40]:
with open("paragraphs.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

# UCSB Old Loop (Ignore)

In [327]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
tracker_df = pd.DataFrame(columns = ["Speech_id",'text','tracker'])

for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            ##################################################
            # Define a pars tracker checking what objects have been used
            pars_tracker = pd.DataFrame(columns = ["Speech_id",'text','tracker'])
            pars_tracker['text'] = pars
            pars_tracker["Speech_id"] = title
            # loop through all paragraphs inside each speech 
            i = 0    
            while i < (len(pars) - 1):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    pars_tracker["tracker"][i] = "used"
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    pars_tracker["tracker"][i] = "used"
                    pars_tracker["tracker"][i+1] = "used"
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1:
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        pars_tracker["tracker"][i] = "used"
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                else: 
                    row = [pars[i - 1] + " " + pars[i], cand, term, title, True]
                    print(i)
                    rows.pop()
                    rows.append(row)
            tracker_df = tracker_df.append(pars_tracker)


In [332]:
### ADD par id
tracker_df = tracker_df[tracker_df['tracker'] != "used"]
tracker_df['par_id'] = "None"
for speech_id in tracker_df['Speech_id']:
    tracker_df.loc[tracker_df['Speech_id'] == speech_id,'par_id'] = list(range(998, 998 + len(tracker_df[tracker_df['Speech_id'] == speech_id])))

In [335]:
tracker_df[tracker_df['tracker'] != "used"].to_csv('unused_UCSB.csv')
tracker_df['par_id'].unique()

array([998, 999], dtype=object)

In [42]:
col_names = ["text","candidate", "term", "title", "comp"]
with open("paragraphs_old.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

In [43]:
paragraphs_new = pd.read_csv("paragraphs.csv")
paragraphs_old = pd.read_csv("paragraphs_old.csv")

In [59]:
# compare the old and the new one
df = pd.merge(paragraphs_old, paragraphs_new, on="text", how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)


In [67]:
print(len(paragraphs_new))
print(len(paragraphs_old))
df.to_csv("paragraphs_dropped.csv")

# Repeat the process for Annenberg data

In [15]:
import pandas as pd
annen_meta = pd.read_csv("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/annenberg_metadata.csv")

In [29]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 

for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        Type = annen_meta[annen_meta['id_speech'] == float(id_speech)]['type'].iloc[0] 
        if Type == 'Speeches': #only keep speeches 
        # the last paragraph is not part of body text
            pars = [i for i in pars if i] #remove empty strings 
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
            pars = [re.sub("\t", "",par) for par in pars]
            pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
            # loop through all paragraphs inside each speech  
            i = 0    
            while i < len(pars):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [id_speech, pars[i], party, term, False]
                    pars_tracker["tracker"][i] = "used"
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [id_speech, pars[i] + " " + pars[i +1], party, term, True]
                    pars_tracker["tracker"][i] = "used"
                    pars_tracker["tracker"][i+1] = "used"
                    i += 2
                    if len(sent_tokenize(row[1])) < 3 and i < len(pars):
                        row_2 = [id_speech, row[1] + " " + pars[i], party, term,True]
                        pars_tracker["tracker"][i] = "used"
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                elif i == (len(pars) - 1): 
                    row = [id_speech,rows[-1][1] + " " + pars[i], party, term, True]
                    rows.pop()
                    rows.append(row)
                    i += 1  

In [30]:
with open("paragraphs_annenberg.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

# Old Loop (ignore)

In [256]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 
tracker_df = pd.DataFrame(columns = ["Speech_id",'text','tracker'])

for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        Type = annen_meta[annen_meta['id_speech'] == float(id_speech)]['type'].iloc[0] 
        if Type == 'Speeches': #only keep speeches 
            pars = [i for i in pars if i] #remove empty strings 
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
            pars = [re.sub("\t", "",par) for par in pars]
            pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
            ##################################################
            # Define a pars tracker checking what objects have been used
            pars_tracker = pd.DataFrame(columns = ["Speech_id",'text','tracker'])
            pars_tracker['text'] = pars
            pars_tracker["Speech_id"] = id_speech
            # the last paragraph is not part of body text
            del pars[-1] 
            # loop through all paragraphs inside each speech  
            i = 0     
            while i < (len(pars) - 1): 
                n_sent = len(sent_tokenize(pars[i])) 
                if n_sent >2: 
                    row = [id_speech, pars[i], party, term, False] 
                    pars_tracker["tracker"][i] = "used"
                    rows.append(row) 
                    i += 1 
                elif i < len(pars) - 1: 
                    row = [id_speech, pars[i] + " " + pars[i +1], party, term, True] 
                    #Set pars tracker
                    pars_tracker["tracker"][i] = "used"
                    pars_tracker["tracker"][i+1] = "used"
                    i += 2 
                    if len(sent_tokenize(row[1])) < 3 and i < len(pars) - 1: 
                        row_2 = [id_speech, row[1] + " " + pars[i], party, term, True] 
                        pars_tracker["tracker"][i] = "used"
                        i += 1 
                        rows.append(row_2) 
                    else:  
                        rows.append(row) 
                else:  
                    row = [id_speech, pars[i - 1] + " " + pars[i], party, term, True] 
                    rows.pop() 
                    rows.append(row)
            tracker_df = tracker_df.append(pars_tracker)


In [209]:
with open("used_pars.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(["text","used"]) 
    writer.writerows(pars_tracker_total)

In [265]:
# tracker_df[tracker_df['tracker'] != "used"].to_csv('used_tracker.csv')


In [33]:
with open("paragraphs_annenberg_old.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

In [34]:
paragraphs_new = pd.read_csv("paragraphs_annenberg.csv")
paragraphs_old = pd.read_csv("paragraphs_annenberg_old.csv")
# compare the old and the new one
df = pd.merge(paragraphs_old, paragraphs_new, on="text", how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

# merge the dropped paragraphs, too (ignore)

In [374]:
# load the cleaned dropped paragraphs
unused_annenberg_cleaned = pd.read_csv("unused_annenberg_cleaned.csv")
unused_UCSB = pd.read_csv("unused_UCSB.csv")

all_unused = unused_annenberg_cleaned[["Speech_id", "text", "par_id"]].append(unused_UCSB)


In [392]:
all_unused

,Speech_id,text,par_id
0,2682,"But we can turn those policies around, reinvig...",998
1,2682,And we should start by repealing the social se...,999
2,2696,I would like to have your support.,998
3,906,A democracy is a live society--and growth is t...,998
4,906,You will hear a lot about the need for a chang...,999
...,...,...,...
346,2004-09-16-general-conference-las,"Thank you, God bless you, and God bless America.",998
347,2000-10-30-remarks-muskegon-michigan,Thank you — God bless you — and God bless Amer...,998
348,2000-09-29-chevy-chase-maryland,Thank you — God bless you — and let's win this...,998
349,2000-11-01-remarks-kissimmee-florida,Thank you — God bless you — and let's win in F...,998


In [406]:
col_names = ["Speech_id","text","par_id"] 
rows = [] 

for speech in all_unused['Speech_id'].unique():
    pars = all_unused[all_unused['Speech_id'] == speech]
    if len(pars) == 1:
        row = [speech, pars['text'].iloc[0], 998] 
        rows.append(row)
    if len(pars) >1:
        n_sent = len(sent_tokenize(pars['text'].iloc[0])) 
        if n_sent >2: 
            row = [speech, pars['text'].iloc[0], 998] 
            rows.append(row) 
            row2 = [speech, pars['text'].iloc[1], 999] 
            rows.append(row2)
        else: 
            row = [speech, pars["text"].iloc[0] + " " + pars["text"].iloc[1], 998] 
            rows.append(row)


In [407]:
with open("all_unused_merged.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

In [351]:
# # group by speech and aggregate         
# all_unused = all_unused.astype(str).groupby(all_unused.Speech_id, as_index=True).agg(' '.join)
# all_unused['par_id'] = 998 # reset par_id because we don't want those concatenated
# all_unused.to_csv("all_unused_merged.csv")


In [408]:
# merge to get the speech meta data 
# all_pars = pd.read_csv('20201115_all_paragraphs.csv')
all_unused = pd.read_csv("all_unused_merged.csv")
all_unused = all_unused.merge(all_pars[["Speech_id","party", "term"]].drop_duplicates(), how = "left", on = "Speech_id")

all_unused.to_csv("all_unused_merged.csv")


In [409]:
# bind the dropped ones to the main file 
all_pars_added = all_pars[["Speech_id","party", "term", "text","par_id"]].append(all_unused)
# check if there are duplicates
all_pars_added.duplicated(subset = ['Speech_id', "par_id"]).unique()
# write to file
all_pars_added.to_csv("20210329_all_paragraphs.csv")

In [410]:
all_unused.par_id.unique()

array([998, 999])

# Combine UCSB data and Annenberg data


In [43]:
import numpy as np

ucsb = pd.read_csv('paragraphs.csv')
annenberg = pd.read_csv('paragraphs_annenberg.csv',skipinitialspace=True)

# remove \n and \t in annenberg data 
annenberg['text'] = annenberg['text'].replace(r'\n',' ', regex=True) 
annenberg['text'] = annenberg['text'].replace(r'\t',' ', regex=True) 

In [44]:
# create a party variable for ucsb data to match the annenberg data
dems = ['obama', 'gore-jr', 'clinton','kerry']
ucsb['party'] = ['dem' if x in dems else 'rep' for x in ucsb['candidate']] 

# match columns
ucsb_matched = ucsb[['title', 'text', 'party', 'term', 'comp']]
ucsb_matched.rename(columns = {'title':'Speech_id'}, inplace = True) 


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
all_data = annenberg.append(ucsb_matched)

In [142]:
# add paragraph id
all_dt = pd.DataFrame(columns = ['Speech_id', 'text', 'party', 'term', 'comp', 'par_id'])
current_id = all_data['Speech_id'].iloc[0]
par_id = 0
for index, row in all_data.iterrows():
    speech_id = row['Speech_id']
    par_id += 1
    if speech_id != current_id:
        current_id = speech_id
        par_id = 0
    new_row = row
    new_row['par_id'] = par_id
    all_dt = all_dt.append(new_row)
#     print(new_row)
#     speech = all_data[all_data['Speech_id'] == sp_id]
#     for i in range(len(speech)):
#         par = speech.loc[i]
#         par['par_id'] = i + 1
#         #all_dt.append(par)
#         print(par)
#     for ind in speech.index: 
#         i = 1
#         speech['par_id'][ind] = i
#         i += 1
#         print(speech)
#     for index, row in speech.iterrows(): 
#         i = 1
#         row['par_id'] = i
#         i += 1
#         print(row)

In [160]:
all_data.to_csv('all_paragraphs.csv')

# Sample the data (ignore)

In [163]:
import random
random.seed(11)
sample = all_data.sample(n = 1000)
sample1 = sample[0:500]
sample2 = sample[500:]

In [162]:
sample1.to_csv('yuchen_sample.csv')
sample2.to_csv('bart_sample.csv')

In [145]:
all_dt.to_csv('NEW_all_paragraphs.csv')

In [156]:
all_data.index = range(0,len(all_data))
all_data

In [3]:
all_dt = pd.read_csv('20201115_all_paragraphs.csv')
#print(len(all_dt))
sample1 = pd.read_csv('yuchen_sample.csv')
sample2 = pd.read_csv('bart_sample.csv')
sample3 = pd.read_csv('yuchen_2nd_sample.csv')
sample4 = pd.read_csv('bart_2nd_sample.csv')
sample5 = pd.read_csv('oscar_sample.csv')

sample = sample1.append(sample2.append(sample3.append(sample4.append(sample5,ignore_index=True))))
sample['total_id'] = sample['Speech_id'].astype(str)  + '_' + sample['par_id'].astype(str) 
# all_dt['total_id'] = all_dt['Speech_id'].astype(str)  + '_' + all_dt['par_id'].astype(str) 

sample

,Unnamed: 0,Speech_id,text,party,term,comp,Unnamed: 0.1,populist_old_keywords,par_id,total_id
0,15777,2192,The fact of the matter is that we find in the ...,rep,1968,True,NaN,NaN,NaN,2192_nan
1,8283,1987,The first thing you do Tuesday morning is get ...,dem,1964,False,NaN,NaN,NaN,1987_nan
2,50267,3156,"Governor Clinton is talking about ""Well, we re...",rep,1992,False,NaN,NaN,NaN,3156_nan
3,12341,1402,"In the meantime, wages have been raised and th...",rep,1956,False,NaN,NaN,NaN,1402_nan
4,44383,3226,"And when people need it, they're smart enough ...",dem,1996,False,NaN,NaN,NaN,3226_nan
...,...,...,...,...,...,...,...,...,...,...
995,495,2004-09-07-discussion-sedalia-missouri,"The third lesson is, is that we must deal with...",rep,2004,False,NaN,True,120.0,2004-09-07-discussion-sedalia-missouri_120.0
996,496,3171,"So, here we go. Governor Clinton--I honestly b...",rep,1992,False,NaN,True,3.0,3171_3.0
997,497,1829,"This is not the time to go into it in detail, ...",rep,1960,True,NaN,True,6.0,1829_6.0
998,498,2090,My only prayer is that I will have the strengt...,dem,1968,False,NaN,True,14.0,2090_14.0


In [9]:
unsampled_dt = all_dt[~all_dt['total_id'].isin(sample['total_id'])]
unsampled_dt

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id,total_id
0,0,2682,"When we start talking about the economy, it's ...",rep,1980,False,True,1,2682_1
1,1,2682,It's no secret which groups are hit the hardes...,rep,1980,False,False,2,2682_2
2,2,2682,How can our elderly who have worked so hard to...,rep,1980,True,False,3,2682_3
3,3,2682,I believe that social security is one of this ...,rep,1980,True,True,4,2682_4
4,4,2682,"In contrast, I am committed to an economic pro...",rep,1980,True,False,5,2682_5
...,...,...,...,...,...,...,...,...,...
71822,72341,2000-08-17-national-convention-los,"And I ask all of you, my fellow citizens, from...",dem,2000,True,False,84,2000-08-17-national-convention-los_84
71823,72342,2000-08-17-national-convention-los,"I know my own imperfections. For example, I kn...",dem,2000,False,False,85,2000-08-17-national-convention-los_85
71824,72344,2000-08-17-national-convention-los,Vice President Gore. But the presidency is mor...,dem,2000,True,False,87,2000-08-17-national-convention-los_87
71825,72345,2000-08-17-national-convention-los,There are big choices ahead and our whole futu...,dem,2000,True,False,88,2000-08-17-national-convention-los_88


In [15]:
# drop paragraphs already sampled
new_dat = (pd.merge(all_dt, sample[['Speech_id', 'text', 'party', 'term', 'comp']], indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

new_dat

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id
0,0.0,2682,"When we start talking about the economy, it's ...",rep,1980,False,True,1.0
1,1.0,2682,It's no secret which groups are hit the hardes...,rep,1980,False,False,2.0
2,2.0,2682,How can our elderly who have worked so hard to...,rep,1980,True,False,3.0
3,3.0,2682,I believe that social security is one of this ...,rep,1980,True,True,4.0
4,4.0,2682,"In contrast, I am committed to an economic pro...",rep,1980,True,False,5.0
...,...,...,...,...,...,...,...,...
71857,72341.0,2000-08-17-national-convention-los,"And I ask all of you, my fellow citizens, from...",dem,2000,True,False,84.0
71858,72342.0,2000-08-17-national-convention-los,"I know my own imperfections. For example, I kn...",dem,2000,False,False,85.0
71859,72344.0,2000-08-17-national-convention-los,Vice President Gore. But the presidency is mor...,dem,2000,True,False,87.0
71860,72345.0,2000-08-17-national-convention-los,There are big choices ahead and our whole futu...,dem,2000,True,False,88.0


In [120]:
# merge and clean up new dat 
all_dt = pd.read_csv('20201110_all_paragraphs.csv')
merged = new_dat.merge(all_dt, how = "left", on = "text")
merged = merged[merged['Speech_id_x'] == merged['Speech_id_y']] 
del merged['Speech_id_y']

merged = merged.drop_duplicates(subset = ["Speech_id_x", "par_id"])


del merged["Unnamed: 0_x"]
del merged["Unnamed: 0_y"]
del merged["party_y"]
del merged["term_y"]
del merged["comp_y"]

pd.options.display.max_colwidth = 200
merged = merged[merged['text'].notna()] #remove na 
audience = merged[merged['text'].str.contains('Audience')] # check where the audience reaction is 
audience = audience[audience['text'].str.split().str.len()<13] # most audience reactions are short 
merged = (pd.merge(merged, audience, indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

merged.rename(columns = {'Speech_id_x':'Speech_id', 'party_x':"party", "term_x":"term", "comp_x":"comp", "populist_old_keywords_x":"populist_old_keywords"}, inplace = True) 
del merged['populist_old_keywords_y']

In [16]:
# SAMPLE
import random
import pandas as pd
# merged = pd.read_csv('20201115_all_paragraphs.csv')
random.seed(11)
new_sample = new_dat.sample(n = 250)

# # first 500 completely random
# sample1 = sample[0:500]
# # second sample = 250 random + 250 populist keyword == T
# sample2_1 = sample[500:750]

# sample = sample1.append(sample2_1, ignore_index = True)

# # exclude the ones already sampled 
# new_sample = (pd.merge(merged, sample, indicator=True, how='outer')
#          .query('_merge=="left_only"')
#          .drop('_merge', axis=1))
# sample2_2 = new_sample[new_sample['populist_old_keywords'] == True].sample(n = 250)

# sample2 = sample2_1.append(sample2_2, ignore_index = True)

In [24]:
new_sample['par_id'] = new_sample['par_id'].astype(int)
new_sample['total_id'] = new_sample['Speech_id'].astype(str)  + '_' + new_sample['par_id'].astype(str) 
new_sample = new_sample[["total_id",'Speech_id',"par_id", "text", "party", "term", "comp", "populist_old_keywords"]]
#sample1.to_csv('bart_2nd_sample.csv')
#sample2.to_csv('yuchen_2nd_sample.csv')
new_sample.to_csv('oscar_2nd_sample.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
merged[merged['text'].str.contains('Audience')]

,Speech_id_x,text,party_x,term_x,comp_x,par_id
198,2133,"Mr. Nixon has come to this city and he has brought a message on the date of October 23rd which says the ""U. S. World Lead Lost, Nixon Tells Battle Creek Audience,"" and what did he say? He said we...",dem,1968,False,26
15470,2804,"Well now, it's time for me to go-- [Audience: No!] Yes--oh--have to. But I see so many families here this afternoon, so many fine young people. May I just leave you with one last thought from my h...",rep,1984,True,26
50512,2012-09-27-virginia-beach-virginia,The President. Jim Webb! Audience members. Four more years! Four more years! Four more years!,dem,2012,True,0
50518,2012-09-27-virginia-beach-virginia,"Audience members. Yours! The President. No, well, I know some in the crowd may be a little biased, but—[Laughter]—but I also want to speak to the audience who may be seeing this over the television.",dem,2012,True,7
50528,2012-09-27-virginia-beach-virginia,"Number one, I want to export more products and I want to outsource fewer jobs. When my opponent said we should just ""let Detroit go bankrupt""—— Audience members. Boo!",dem,2012,True,18
...,...,...,...,...,...,...
69728,2016-10-12-pavilion-ocala-florida,"And who is going to pay for the wall? Audience: ""Mexico"" 100 percent. They just don't know it yet. They don't know it yet, but 100 percent. They'll pay for the wall.",rep,2016,True,116
71297,2000-08-17-national-convention-los,"You know... Audience. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,31
71298,2000-08-17-national-convention-los,"Vice President Gore. Are you with me? Audience. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,32
71312,2000-08-17-national-convention-los,"And all of this...[applause] Audience. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,48


In [136]:
merged.to_csv('20201115_all_paragraphs.csv')

# Add paragraph id (ignore)

In [20]:
# 
bart_sample = pd.read_csv("bart_sample.csv")
bart_annot = pd.read_csv("annotated_par_populism_bart.csv")
all_dat = pd.read_csv("20201115_all_paragraphs.csv")

#print(bart_sample)
print(bart_annot)
#print(all_dat)

#all_dat[0:50]

     Unnamed: 0                                               text  \
0             0  The future is my responsibility and it's yours...   
1             1  I ask you to help us make new history and I as...   
2             2  But, getting back to the issues before us, I s...   
3             3  It used to be in America all the racial issues...   
4             4  Mr. Lamb. Yes, it's a legal term. The Presiden...   
..          ...                                                ...   
492         766  Ten months ago, in a tragic moment, I was call...   
493         767  He says he cares about the middle class, but h...   
494         768  We have got to have a tough plan to bring heal...   
495         769  The constant partisan rancor that stops us fro...   
496         770  We began working on an energy policy, which th...   

            label   annotated_at                               Speech_id_x  \
0    not populist  11/4/20 15:28                                      2600   
1  

In [23]:
bart_annot = bart_annot[['Speech_id_x', 'par_id_y', 'text', 'party_x','term_x','comp_x','label']]

cols = ['Speech_id','par_id','text', 'party','term','comp','label']
 
bart_annot.columns = cols



KeyError: "['Speech_id_x', 'comp_x', 'party_x', 'term_x', 'par_id_y'] not in index"

In [24]:
print(bart_annot)
bart_annot.to_csv("bart_annotated.csv")

                                    Speech_id  par_id  \
0                                        2600     8.0   
1                                        2005    27.0   
2                                        1871     3.0   
3                                        3256     9.0   
4      2004-09-07-discussion-sedalia-missouri    47.0   
..                                        ...     ...   
492                                      1922    21.0   
493                                      2868    14.0   
494                                      3004    31.0   
495  2008-10-01-remarks-independence-missouri    16.0   
496                                      2575    11.0   

                                                  text party  term   comp  \
0    The future is my responsibility and it's yours...   dem  1980  False   
1    I ask you to help us make new history and I as...   dem  1968   True   
2    But, getting back to the issues before us, I s...   rep  1960   True   
3    It

In [13]:
#### add par_id to Yuchen first sample


oscar_sample = pd.read_csv('oscar_sample.csv')


oscar_sample['drawn'] = ['randomly'] *250 + ['keyword'] * 250

oscar_sample

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id,drawn
0,0,2162,But don't worry about that. You know who is in...,rep,1968,False,False,3,randomly
1,1,2016-10-28-manchester-new-hampshire,And these people are seriously corrupt. Terrib...,rep,2016,True,True,21,randomly
2,2,1045,We can make our society ever more productive a...,dem,1952,True,False,13,randomly
3,3,2397,Q. Do you think that Gerald Ford had anything ...,dem,1976,True,False,30,randomly
4,4,2012-10-05-rally-fairfax-virginia,One of the main reasons record surpluses under...,dem,2012,False,False,9,randomly
...,...,...,...,...,...,...,...,...,...
495,495,2004-09-07-discussion-sedalia-missouri,"The third lesson is, is that we must deal with...",rep,2004,False,True,120,keyword
496,496,3171,"So, here we go. Governor Clinton--I honestly b...",rep,1992,False,True,3,keyword
497,497,1829,"This is not the time to go into it in detail, ...",rep,1960,True,True,6,keyword
498,498,2090,My only prayer is that I will have the strengt...,dem,1968,False,True,14,keyword


In [10]:
yuchen_sample = yuchen_annot.drop('label', axis = 1)


,Unnamed: 0,Speech_id,par_id,text,party,term,comp,drawn
0,0,2192,27,The fact of the matter is that we find in the ...,rep,1968,True,randomly
1,1,1987,25,The first thing you do Tuesday morning is get ...,dem,1964,False,randomly
2,2,3156,11,"Governor Clinton is talking about ""Well, we re...",rep,1992,False,randomly
3,3,1402,18,"In the meantime, wages have been raised and th...",rep,1956,False,randomly
4,4,3226,41,"And when people need it, they're smart enough ...",dem,1996,False,randomly
...,...,...,...,...,...,...,...,...
490,490,2004-10-26-dubuque-iowa-0,34,You cannot be pro-doctor and pro-patient and p...,rep,2004,False,randomly
491,491,1076,4,"We are not only, as I say, a moral countr...",dem,1952,False,randomly
492,492,1659,8,The housing--if there is any state in the unio...,dem,1960,False,randomly
493,493,2008-11-03-jacksonville-florida-1,31,Don't believe for a second this election is ov...,dem,2008,False,randomly


# Remove All Caps Tags (ignore)

In [281]:
annenberg = pd.read_csv("used_tracker.csv")

In [129]:
#### OBSELETE
# for i in range(len(annenberg['text'])):
#     s = annenberg['text'][i]
#     s = s.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
#     all_caps = "".join(re.findall(r"[A-Z]{3,}.*$", s))# extract caps (start from the end, greedy search for all caps of 3 or more) 
#     annenberg['text'][i] = re.sub(all_caps, '', s)
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [282]:
for i in range(len(annenberg['text'])):
    try:
        annenberg['text'][i] = re.match(r'^.+?(?=[A-Z]{3,})', annenberg['text'][i]).group()
    except:
        continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [283]:
#### Drop empties and <3 letters 
annenberg = annenberg[annenberg["text"].notna()]
annenberg["length"]=annenberg["text"].apply(lambda x: len(x))
annenberg = annenberg[annenberg["length"]> 2]

In [310]:
### ADD par id
annenberg['par_id'] = "None"
for speech_id in annenberg['Speech_id']:
    annenberg.loc[annenberg['Speech_id'] == speech_id,'par_id'] = list(range(998, 998 + len(annenberg.loc[annenberg['Speech_id'] == speech_id])))

In [312]:
annenberg.to_csv('used_tracker_cleaned.csv')

In [300]:
range(998, 998+2)

range(998, 1000)

In [311]:
annenberg

,Unnamed: 0,Speech_id,text,tracker,length,par_id
0,13,2682,"But we can turn those policies around, reinvig...",NaN,103,998
1,14,2682,And we should start by repealing the social se...,NaN,73,999
2,33,2696,I would like to have your support.,NaN,35,998
5,15,906,A democracy is a live society--and growth is t...,NaN,503,998
6,16,906,You will hear a lot about the need for a chang...,NaN,216,999
...,...,...,...,...,...,...
4316,35,1386,We need a Democratic administration in Washing...,NaN,155,998
4318,61,2857,And now I'll just finish by saying something t...,NaN,145,998
4319,62,2857,Thank you. Thank you all very much. Thank you.,NaN,46,999
4322,13,2843,Today we can talk and negotiate in confidence ...,NaN,184,998


In [274]:
type(annenberg['text'])

pandas.core.series.Series

# Remove Q&A and Audience Speech

In [3]:
# load the paragraphs (with 2020)
all_paras = pd.read_csv("20220115_all_paragraphs_2020_added.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
all_paras.head()

,Unnamed: 0,X,Speech_id,text,party,term,comp,populist_old_keywords,par_id,speech_par_id
0,1,0,2682,"When we start talking about the economy, it's ...",rep,1980,False,True,1,2682_1
1,2,1,2682,It's no secret which groups are hit the hardes...,rep,1980,False,False,2,2682_2
2,3,2,2682,How can our elderly who have worked so hard to...,rep,1980,True,False,3,2682_3
3,4,3,2682,I believe that social security is one of this ...,rep,1980,True,True,4,2682_4
4,5,4,2682,"In contrast, I am committed to an economic pro...",rep,1980,True,False,5,2682_5


Things to get rid of

Q&A through Capital word with :

Exception for "Americans:"

Audience: or Audience. 


In [75]:
junk_qa = []
for i in range(len(all_paras['text'])):
    if re.search('Audience\.|AUDIENCE\.|Audience member\.|Audience Member\.|Q\.|QUESTION\.|Question\.', all_paras['text'][i]) is not None:
        junk_qa.append(all_paras.loc[i])
    elif re.search('[A-Z][A-Za-z]*:', all_paras['text'][i]) is not None and re.search("Americans:", all_paras['text'][i]) is None:
        junk_qa.append(all_paras.loc[i])


In [76]:
junk_qa_df = pd.DataFrame(junk_qa)

In [82]:
junk_qa_df.to_csv("junk_text.csv")
junk_qa_df.iloc[0:1500].to_csv("junk_text_1.csv")
junk_qa_df.iloc[1500:2770].to_csv("junk_text_2.csv")

In [63]:
re.search('Audience\.|AUDIENCE\.',"Audience. Thank you, George. Thank you, George. Thank you, George.")

<re.Match object; span=(0, 9), match='Audience.'>

In [43]:
len(junk_qa_df['Speech_id'].unique())

757

In [78]:
len(junk_qa_df)

2770

In [28]:
for i in range(len(all_paras['text'])):
    all_paras['length'][i] = len(all_paras['text'][i].split())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
junk_qa_df.iloc[0:1500]

,Unnamed: 0,X,Speech_id,text,party,term,comp,populist_old_keywords,par_id,speech_par_id,length
177,178,177,2133,"And that choir, or that singing and choral gro...",dem,1968,True,False,1,2133_1,36
179,180,179,2133,VOICES: We Want Humphrey! VICE PRESIDENT HUMP...,dem,1968,True,False,3,2133_3,51
184,185,184,2133,VOICES: We want Humphrey! VICE PRESIDENT HU...,dem,1968,True,False,8,2133_8,82
185,186,185,2133,VOICES: You! You! VICE PRESIDENT HUMPHREY: ...,dem,1968,True,False,9,2133_9,26
186,187,186,2133,VOICES: Humphrey! Humphrey! VICE PRESIDENT H...,dem,1968,True,False,10,2133_10,21
...,...,...,...,...,...,...,...,...,...,...,...
46902,46903,46902,2505,"Q. My question for you today, sir, is, how do ...",rep,1976,True,False,6,2505_6,81
46903,46904,46903,2505,"Q. Washington, Oregon, and Montana are three s...",rep,1976,True,False,7,2505_7,63
46906,46907,46906,2505,"Q. First, I commend you on being able to prono...",rep,1976,False,False,10,2505_10,91
46910,46911,46910,2505,"Q. If we can have the burden, we'll take the r...",rep,1976,True,False,14,2505_14,89


In [83]:
qa = []
for i in range(len(all_paras['text'])):
    if re.search('\. Q\.|Q\.|QUESTION\.|Question\.', all_paras['text'][i]) is not None:
        qa.append(all_paras.loc[i])

In [89]:
qa_df = pd.DataFrame(qa)
qa_speech = qa_df['Speech_id'].unique()
qa_speech_df = pd.DataFrame(qa_speech)
qa_speech_df.to_csv("q&a_speeches.csv")

In [91]:
qa_speech_df = pd.read_csv("q&a_speeches.csv")

In [92]:
qa_speech_df

,Speech_id
0,3005
1,2455
2,2874
3,2450
4,2487
5,3003
6,1402
7,2839
8,2781
9,2794


In [105]:
junk_qa_df_1 = junk_qa_df.iloc[0:1500]
junk_qa_df_1_cleaned = junk_qa_df_1[~junk_qa_df_1['Speech_id'].isin(qa_speech_df["Speech_id"])]

In [103]:
junk_qa_df_1['Speech_id'].isin(qa_speech_df["Speech_id"])


177      False
179      False
184      False
185      False
186      False
         ...  
46902     True
46903     True
46906     True
46910     True
46912     True
Name: Speech_id, Length: 1500, dtype: bool

In [108]:
junk_qa_df_1_cleaned.to_csv("junk_text_1.csv")

In [109]:
junk_qa_df_1_cleaned

,Unnamed: 0,X,Speech_id,text,party,term,comp,populist_old_keywords,par_id,speech_par_id,length
177,178,177,2133,"And that choir, or that singing and choral gro...",dem,1968,True,False,1,2133_1,36
179,180,179,2133,VOICES: We Want Humphrey! VICE PRESIDENT HUMP...,dem,1968,True,False,3,2133_3,51
184,185,184,2133,VOICES: We want Humphrey! VICE PRESIDENT HU...,dem,1968,True,False,8,2133_8,82
185,186,185,2133,VOICES: You! You! VICE PRESIDENT HUMPHREY: ...,dem,1968,True,False,9,2133_9,26
186,187,186,2133,VOICES: Humphrey! Humphrey! VICE PRESIDENT H...,dem,1968,True,False,10,2133_10,21
...,...,...,...,...,...,...,...,...,...,...,...
46575,46576,46575,2707,"THE TRUTH ABOUT OUR FUTURE: Second, I believe ...",dem,1984,True,False,15,2707_15,77
46582,46583,46582,2707,"NEW POPULISM: Third, Mr. Reagan and I differ o...",dem,1984,True,False,22,2707_22,40
46588,46589,46588,2707,"ARMS CONTROL: Finally, Mr. Reagan and I disagr...",dem,1984,True,False,28,2707_28,82
46591,46592,46591,2707,Mr. Reagan is the first president since Hoover...,dem,1984,True,False,31,2707_31,53
